---
# Asymmetric FOD estimation tutorial

**Matteo Bastiani, FMRIB**

May 2018

---

This jupyter notebook explaines how to estimate asymmetric FODs from dMRI data using the qboot_v2 python package.
Both FSL and the qboot_v2 python package need to have been already downloaded and succesfully installed.

In this tutorial, we will use dMRI data provided by the FSL course.
Data can be donwloaded from the following link:
https://fsl.fmrib.ox.ac.uk/fslcourse/downloads/fdt.tar.gz

Once the download is complete, unpack the fdt.tar.gz file and change the following fsl_data_path and tutorial path accordingly.

In [1]:
fsl_data_path = '/Users/matteob/Downloads/fdt/fdt2/subj1'
tutorial_path = '/Users/matteob/qboot_v2/qboot_v2/tutorial'

In [2]:
%%bash -s "{fsl_data_path}"

mkdir -p ${1}/dti
${FSLDIR}/bin/dtifit -k ${1}/data.nii.gz -o ${1}/dti/dti -m ${1}/nodif_brain_mask -r ${1}/bvecs -b ${1}/bvals  --kurt

${FSLDIR}/bin/fslmaths ${1}/dti/dti_FA.nii.gz -thr 0.4 -ero -bin ${1}/dti/high_fa_mask

0 116 0 116 0 76
0 slices processed
1 slices processed
2 slices processed
3 slices processed
4 slices processed
5 slices processed
6 slices processed
7 slices processed
8 slices processed
9 slices processed
10 slices processed
11 slices processed
12 slices processed
13 slices processed
14 slices processed
15 slices processed
16 slices processed
17 slices processed
18 slices processed
19 slices processed
20 slices processed
21 slices processed
22 slices processed
23 slices processed
24 slices processed
25 slices processed
26 slices processed
27 slices processed
28 slices processed
29 slices processed
30 slices processed
31 slices processed
32 slices processed
33 slices processed
34 slices processed
35 slices processed
36 slices processed
37 slices processed
38 slices processed
39 slices processed
40 slices processed
41 slices processed
42 slices processed
43 slices processed
44 slices processed
45 slices processed
46 slices processed
47 slices processed
48 slices processed
49 slices pro

In [3]:
# Import the csdeconv module
from qboot_v2.csdeconv import csdeconv as csd

## White matter response function estimation

In [4]:
# Estimate the white matter response function from high FA voxels up to spherical harmoic order of 4
resp_max_harmonic_order = 4

r_wm = csd.Response.get_response(fsl_data_path + '/data.nii.gz',
                              fsl_data_path + '/dti/high_fa_mask.nii.gz',
                              fsl_data_path + '/bvals',
                              fsl_data_path + '/bvecs',
                              resp_max_harmonic_order,
                              dti_basename = fsl_data_path + '/dti/dti')

Found 255 masked voxels
Found 4 shells


The response function estimation method has correctly identified the 4 different b-shells in the dataset (0, 500, 1500, 2500 s/mm^2). It has estimated one response function per b-shell and stored its spherical harmonics coefficients in a [N_shells X N_even_coefficients] array.

We can save the coefficients to a text file for reference and future use:

In [5]:
# Store the response coefficients to a text file
r_wm.write_coefficients(fsl_data_path + '/response_wm_l4.txt')

## Estimate aFODs

Now, we can estimate the aFODS using the estimated response function.

In [10]:
# To speed things up, let's mask only 3 coronal slices...
import nibabel as nib

mask_obj = nib.load(fsl_data_path + '/nodif_brain_mask.nii.gz')
mask = mask_obj.get_data()

mask[:, 0:56, :] = 0
mask[:, 59:, :] = 0

nib.Nifti1Image(mask, None, mask_obj.header).to_filename(fsl_data_path + '/nodif_brain_mask_roi.nii.gz')

In [11]:
# Estimate FODs up to harmonic order 8
max_fod_harmonic_order = 8

In [12]:
afod = csd.csdeconv(r_wm,
                    fsl_data_path + '/data.nii.gz',
                    fsl_data_path + '/nodif_brain_mask_roi.nii.gz',
                    fsl_data_path + '/bvals', 
                    fsl_data_path + '/bvecs',
                    max_fod_harmonic_order,
                    sym = False,
                    out_file = fsl_data_path + '/MS_afod.nii.gz')

Running SD
Running CSD (using 8 processes)


100% (11815 of 11815) |##################| Elapsed Time: 0:00:39 Time:  0:00:39


Storing SH coefficients


For comparison, we can also estimate the symmetric FODs using multi-shell CSD.

In [13]:
sfod = csd.csdeconv(r_wm,
                    fsl_data_path + '/data.nii.gz',
                    fsl_data_path + '/nodif_brain_mask_roi.nii.gz',
                    fsl_data_path + '/bvals', 
                    fsl_data_path + '/bvecs',
                    max_fod_harmonic_order,
                    sym = True,
                    out_file = fsl_data_path + '/MS_sfod.nii.gz')

Running CSD (using 8 processes)


100% (11815 of 11815) |##################| Elapsed Time: 0:00:36 Time:  0:00:36


Storing SH coefficients


Once both asymmetric and symmetric FODs fit have run, it is possible to visualise the results using FSLeyes.

In [ ]:
%%bash -s "{fsl_data_path}"

fsleyes ${1}/dti/dti_FA.nii.gz -dr 0 0.5 -in spline \
        ${1}/MS_afod.nii.gz -ot sh -sr 10 \
        ${1}/MS_sfod.nii.gz -ot sh -sr 10 &


## Peaks extraction

To extract peaks from the FODs, run the following commands:

In [14]:
import os.path as op
import numpy as np
from qboot_v2.utils.utils import get_peaks

In [15]:
# Read the sphere text file; this contains:
# [x1 y1 z1 n11 n12 n13 -1]
# [x2 y2 z2 n21 n22 n23 n24]
# [x3 y3 z3 n31 n32 n33 n34]
# [...]
# [xN yN zN nN1 nN2 nN3 nN4]
# Where x, y and z are the vertex coordinates and n are the neighbouring vertices index (-1 if none).
resource_dir = op.dirname(csd.__file__)
ico5 = op.join(resource_dir, 'ico_5.txt')

sphere = np.genfromtxt(ico5, dtype=np.float)

In [16]:
# Extract up to 6 peaks from aFODs using non-linear optimisation.
# Note that we need to extract so many peaks beacuse of the FOD being asymmetric.
# In the case of symmetric FODs, 3 would be enough
n_peaks = 6

peaks, amplitudes = get_peaks(sphere,
                              fsl_data_path + '/MS_afod.nii.gz',
                              fsl_data_path + '/nodif_brain_mask_roi.nii.gz',
                              max_fod_harmonic_order, 
                              sym=False, 
                              save_results=True, 
                              n=n_peaks, 
                              non_lin=True)

Storing peaks and amplitudes


Peaks extracted from aFODs can be visualised in FSLeyes, specifying that they should be interpreted as directed.

In [17]:
%%bash -s "{fsl_data_path}"

fsleyes ${1}/dti/dti_FA.nii.gz -dr 0 0.5 -in spline \
        ${1}/MS_afod.nii.gz -ot sh -sr 10 \
        ${1}/peak1.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 \
        ${1}/peak2.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 \
        ${1}/peak3.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 \
        ${1}/peak4.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 \
        ${1}/peak5.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 \
        ${1}/peak6.nii.gz -ot linevector -ld  -xc 1 1 1 -yc 1 1 1 -zc 1 1 1 -lw 2 &


18:32:01: Debug: Adding duplicate image handler for 'Windows bitmap file'
/usr/local/fsl/bin/fsleyes: line 6:  3766 Segmentation fault: 11  ${FSLDIR}/bin/FSLeyes.app/Contents/MacOS/fsleyes $@


## Multi-tissue aFODs

It is possible to specify multiple response functions, one for each tissu-type.
Response function can be estimated as before, i.e., providing a binary mask obtained from, e.g., a random seleciton of  voxels from a gray matter or csf segmentation.

For the purpose of this tutorial, we have included the estimated 0-th order spherical harmonics coefficients for gray matter and csf in the tutorial folder. We can load them using the following commands:

In [18]:
r_gm = csd.Response.read_coefficients(tutorial_path + '/response_gm_l0.txt')

r_csf = csd.Response.read_coefficients(tutorial_path + '/response_csf_l0.txt')

Importing response function coefficients...
4 b-shells detected, max harmonic order=0
Importing response function coefficients...
4 b-shells detected, max harmonic order=0


Now we can run the aFOD estimation commands including the different response functions.

**NOTE**
The function assumes that the first reponse function in the list should always be the white matter one!!!

In [19]:
# We need to tell the function the harmonic orders of the different tissue-specific FODs
max_fod_harmonic_order_mt = [8, 0, 0]

afod = csd.csdeconv([r_wm, r_gm, r_csf],
                    fsl_data_path + '/data.nii.gz',
                    fsl_data_path + '/nodif_brain_mask_roi.nii.gz',
                    fsl_data_path + '/bvals', 
                    fsl_data_path + '/bvecs',
                    max_fod_harmonic_order_mt,
                    sym = False,
                    out_file = fsl_data_path + '/MSMT_afod.nii.gz')

Running SD
Running CSD (using 8 processes)


100% (11815 of 11815) |##################| Elapsed Time: 0:01:34 Time:  0:01:34


Storing SH coefficients
